In [ ]:
from bs4 import BeautifulSoup
import urllib
import json
import pandas as pd
import requests
import time 
import numpy as np


# Dictionary created in order to fill in empty rows in the Dataframe when the user profile queried does not exist

exc = {
    'num_collection': np.nan, 
    'seller_rating_stars': np.nan, 
    'buyer_num_ratings': np.nan,
    'collection_folders_url': np.nan, 
    'banner_url': np.nan, 
    'home_page': np.nan, 
    'avatar_url': np.nan,
    'rating_avg': np.nan,
    'name': np.nan, 
    'buyer_rating': np.nan, 
    'releases_rated': np.nan,
    'seller_num_ratings': np.nan,
    'num_for_sale': np.nan, 
    'curr_abbr': np.nan, 
    'profile': np.nan, 
    'registered': np.nan, 
    'location': np.nan,
    'inventory_url': np.nan, 
    'rank': np.nan,
    'num_lists': np.nan,
    'num_pending': np.nan,
    'collection_fields_url': np.nan,
    'buyer_rating_stars': np.nan,
    'wantlist_url': np.nan, 
    'uri': np.nan,
    'seller_rating': np.nan, 
    'id': np.nan,
    'releases_contributed': np.nan,
    'resource_url': np.nan, 
    'username': np.nan, 
    'num_wantlist': np.nan, 
    'message': 'Request Timeout',
}

# creation of a Dataframe where data is parsed to

col = ['num_collection', 'seller_rating_stars', 'buyer_num_ratings',
       'collection_folders_url', 'banner_url', 'home_page', 'avatar_url',
       'rating_avg', 'name', 'buyer_rating', 'releases_rated',
       'seller_num_ratings', 'num_for_sale', 'curr_abbr', 'profile',
       'registered', 'location', 'inventory_url', 'rank', 'num_lists',
       'num_pending', 'collection_fields_url', 'buyer_rating_stars',
       'wantlist_url', 'uri', 'seller_rating', 'id', 'releases_contributed',
       'resource_url', 'username', 'num_wantlist', 'message']

df1= pd.DataFrame(columns = col)
df1.to_csv('export1.csv')

# creation of a Dataframe to keep track of user profiles that were not queried properly (time out or profile does not exist)

dftimeout1 = pd.DataFrame(columns = ['url'])

# definition of function to get user names, query their profile with the API and parse results in the Dataframe

def get_users(page):
    
    # creation of the username list scraping from the contributors page (discogs.com/contributors)
    
    list_users = []
    url = 'https://www.discogs.com/stats/contributors?page=%s' % page
    try: 
        r = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(r, 'html.parser')
        letters = soup.find_all("a", class_='linked_username')
        for i in range(0,len(letters)):
            list_users.append(['https://api.discogs.com'+letters[i]['href'].replace('user','users'),page])
        
        # querying all the user names collected
        
        for j in range(0, len(list_users)):
            url = list_users[j][0]
            headers = {
            # Authorisation details can be found on discogs.com/developers/
                'Authorization': 'Discogs key=[key], secret=[secret]',
            }
            try:
                r = requests.get(url, headers = headers)
                store = json.loads(r.text)
                
            # exception for timeout or value errors of the API call
            
            except (requests.ConnectionError, ValueError): 
                store = exc
                print('Request Timeout')
            print(list_users[j][0])
            
            # parsing the data in the Dataframe
            
            for l in range(0,len(list(store.keys()))):
                df1.loc[list_users[j][0],list(store.keys())[l]] = list(store.values())[l]
            print(df1.loc[list_users[j][0],'username'], df1.loc[list_users[j][0],'message'])
        print("%s users added from page %s. Total is at %s or %s percent" % (len(letters), page, page*50, (page*50/364421)*100))
    
    # excption for time out error on the scraping 
    
    except urllib.error.URLError: 
        print('URL Timeout')
        dftimeout1.loc[len(dftimeout1)] = url
        dftimeout1.to_csv('timeout1.csv')


# defining the range where the data is scraped from on discogs.com/contributors

for i in range (1,2000): 
    df1.from_csv('export1.csv')
    get_users(i)
    df1.to_csv('export1.csv')
    time.sleep(20) 

    